In [30]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("NORD_Task") \
      .getOrCreate() 
sc = spark.sparkContext




hadoop_conf=sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")

S3_BUCKET = 's3-nord-challenge-data'
S3_REGION = 'eu-central-1'
hadoop_conf.set("fs.s3a.endpoint", f"s3.{S3_REGION}.amazonaws.com")
# see http://blog.encomiabile.it/2015/10/29/apache-spark-amazon-s3-and-apache-mesos/
# hadoop_conf.set("fs.s3a.connection.maximum", "100000")


#number of files to process - will be read as input
N = 100

### Note 
In general I can see 2 approaches to load files data:
   - Approach 1.
     - `spark.read.format('binaryFile').option("pathGlobFilter","<path-glob>").load(<s3-bucket>)`. This solution would read all files with metadata into single DataFrame (path, mod time,  length, content)
     - parse content of file in apropriate resulted dataframe transformation
   - The advantage of it is that you receive parallelized DataFrame, content of file would be read in lazy way during processing each file. So in theory on a big enough spark cluster spark should take care of distributing and performance for you. The problem seems to be when you have to work with pretty read milions of files with unknown file size. You may end up huge memory and performance issues. This problem is shown e.g. in [this blog article](https://wrightturn.wordpress.com/2015/07/22/getting-spark-data-from-aws-s3-using-boto-and-pyspark/). Although it's pretty old I did not find any more recent solution to the issue. It also describes second approach.
   - Approach 2.
       - list all objects you're interested files in s3 bucket into some collection (but without parallelizing it)
       - create parallelized dataframe based on the given collection
       - read and process file content as part of transformations
   - The bottleneck might that you have to iterate over millions of files so the size of the collection to be processed (on one node) might be huge. 
   
As I am not able to test on a large set of data and big enough spark cluster which approach is more efficient. I am going to use approach described in [mentioned article](https://wrightturn.wordpress.com/2015/07/22/getting-spark-data-from-aws-s3-using-boto-and-pyspark). However instead of using boto3 for listing all objects I will use [`hadoop.fs.path.getFilesystem.globStatus`](https://stackoverflow.com/a/67050173/2018369) because boto3 [seems to be not the most effective way](https://stackoverflow.com/q/69920805/2018369) to get file list.

I was also considering one more approach, which however I could not find any good way to implement. So my idea was to create a dataframe similar to the one created by `spark.read.format('binaryFile').option("pathGlobFilter","<path-glob>").load(<s3-bucket>)`, but which contain only prefix of file (first 1024 or 2048 bytes). This way we could have a Dataframe(path, mod time,  length, PE headers), we could process the header of file to get all required PE metadata apart from imports/expors and in next step we could load apropriate sections of file to get imports/exports.


In [32]:
clean_path = '/0/*.???'
malware_path = '/1/*.???'



cleanPath = sc._jvm.org.apache.hadoop.fs.Path(f's3a://{S3_BUCKET}{clean_path}')
cFs = cleanPath.getFileSystem(hadoop_conf)
clean_files = cFs.globStatus(cleanPath)

malwarePath = sc._jvm.org.apache.hadoop.fs.Path(f's3a://{S3_BUCKET}{malware_path}')
mFs = malwarePath.getFileSystem(hadoop_conf)
malware_files = mFs.globStatus(malwarePath)

In [33]:
import random
print(len(malware_files))
files_to_process = random.sample(clean_files, int(N/2))+ random.sample(malware_files, int(N/2))

14652


In [34]:
stat = clean_files[0]
print(stat.getPath().toUri().getRawPath())
print(stat.getPath().getName())
print(stat.getLen())

/0/00ELuByj9iSRf5Rx11Ypl15N6kS2FXmW.dll
00ELuByj9iSRf5Rx11Ypl15N6kS2FXmW.dll
3919


In [35]:
# put files into dataFrame

from pyspark.sql.types import StructType,StructField, StringType
schema = StructType([       
    StructField('path', StringType(), True),
    StructField('size', StringType(), True),
    StructField('type', StringType(), True)
])
data = [(f.getPath().toUri().getRawPath(), f.getLen(), f.getPath().getName().split('.')[-1]) for f in files_to_process]
filesDF= spark.createDataFrame(data=data, schema = schema)
filesDF.show()

+--------------------+-------+----+
|                path|   size|type|
+--------------------+-------+----+
|/0/8pvVKQTZmTNocd...|  47136| dll|
|/0/o5S7zV0fdaIQYq...| 857600| dll|
|/0/AbSwmPGR2df8yr...|  26112| dll|
|/0/MvrYZxvDESOWEr...|3082240| dll|
|/0/PCbTjYBFZv1rZf...|  15360| dll|
|/0/DVoQcVfZEqlf93...|1101312| dll|
|/0/ZplwYyZ8NwfdOE...|  18024| dll|
|/0/9u0YliRw1IM0k5...|3752448| dll|
|/0/XNxscQRlfl7gah...|  72192| dll|
|/0/68XSgKiNv2Eqbs...|   8704| dll|
|/0/25sUeWW4nwWzU3...|  66560| dll|
|/0/dhuK7vFrrugBGA...| 155704| exe|
|/0/py9KkwB8imIL6t...|   8798| dll|
|/0/M15mkckoysbcvo...| 159848| dll|
|/0/VgNRZmVAKIhA1U...|1021440| dll|
|/0/owlX4MqUKHYCki...|  83768| dll|
|/0/K03opjd8veInlu...| 380632| exe|
|/0/vIr1ypSDoURL4f...|   7680| DLL|
|/0/gw16Dgf8gYupMR...| 280576| dll|
|/0/eDA2UwtJcXTqyE...|  33280| dll|
+--------------------+-------+----+
only showing top 20 rows



In [36]:
#process files 

import main
#import importlib
#importlib.reload(main)

reader = main.S3FileReader(S3_BUCKET, S3_REGION)

schema = StructType(filesDF.schema.fields+[
    StructField('architecture', StringType(), True),
    StructField('imports', StringType(), True),
    StructField('exports', StringType(), True)
])
#TODO: This should be fixed
parsed=filesDF.rdd.map(lambda x: (*x, *main.PeParser(x['path'], reader.get_file_stream(x['path']),x['size']).get_short_meta()))

parsedDF = parsed.toDF(schema)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/pyspark/serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/usr/local/lib/python3.10/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/usr/local/lib/python3.10/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 563, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.lock' object


PicklingError: Could not serialize object: TypeError: cannot pickle '_thread.lock' object

In [ ]:
#TODO: store to database